# Static Q-Balls
#### Computation with FastAI

Self-gravitating complex scalar field in flat spacetime with a self-interacting potential, $U(\Psi) = \lambda \left( \Psi^6 - a \Psi^4 + b \Psi^2 \right)$.

*Ansatz*: $\Psi = f(r)e^{i \omega t}$

*Equation of Motion*: $f'' + \dfrac{2}{r} f' + \omega^2 f - \lambda \left( 6 f^5 - 4 a f^3 + 2 b f \right) = 0$

*Contants of the problem*: $\lambda = 1~,~~a = 2~,~~b = 1.1$

##### Import packages

In [1]:
from fastai.basics import *
import numpy as np
import matplotlib.pyplot as plt

/Users/jorgedelgado/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


##### Select device

In [2]:
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')

##### Input data

In [9]:
# Tensor with the input parameter: radial coordinate r
nPoints = 10000
r = torch.rand(nPoints,device=device)

# Select the training and validation data
# Since r is already random, we simply separate the tensor
nTraining = int(0.8*nPoints)
r_training = r[:nTraining]
r_validation = r[nTraining:]

##### Output data

In [12]:
# Tensor with the output of the network that will contain the function the solves the ODE
# We do not know the profile of f, therefore we only construct a tensor with the same size as r filled with zeros
f = torch.zeros_like(r)



In [13]:
f

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='mps:0')

In [39]:
r_training[7999], r_validation[0]

(tensor(0.5236, device='mps:0'), tensor(0.7801, device='mps:0'))

In [19]:
x[5]

tensor(0.2908, device='mps:0')